# Feature Engineering Avanzado
# Preparación de Variables Predictivas para Series de Tiempo

---

## 📊 Objetivo

Este cuaderno implementa **ingeniería de características sofisticada** para maximizar la capacidad predictiva de los modelos de series de tiempo, considerando las particularidades del recaudo identificadas en el EDA:

### Features a Crear:
1. 🔄 **Variables de Rezago (Lag Features)**: Capturar dinámica Diciembre→Enero
2. 📅 **Regresores Exógenos**: Calendario tributario (Renta, ICA)
3. 🕐 **Features Temporales**: Estacionalidad, tendencias, ciclos
4. 📊 **Features Estadísticos**: Rolling means, volatilidad, momentum
5. 🎯 **Features de Interacción**: Combinaciones no lineales

---

**Autor**: Sistema de Análisis Predictivo  
**Fecha**: Febrero 2026  
**Versión**: 1.0

## 1. Configuración y Carga de Datos Depurados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Bibliotecas cargadas")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Cargar datos depurados del cuaderno anterior
# TODO: Ajustar según output del cuaderno 01
print("📁 Cargando datos depurados...")
print("⏳ Pendiente: Implementar después de ejecutar 01_EDA_Depuracion.ipynb")

## 2. Preparación de Serie Temporal Base

Crear dataset con frecuencia mensual, asegurando continuidad temporal

In [ ]:
# TODO: Implementar agregación mensual
print("⏳ Implementar:")
print("1. Crear columna de fecha (año-mes)")
print("2. Agregar recaudo por mes")
print("3. Rellenar gaps temporales si existen")
print("4. Ordenar por fecha ascendente")

## 3. Variables de Rezago (Lag Features)

**Hipótesis clave**: El recaudo de enero refleja la actividad económica de diciembre

In [ ]:
def crear_lags(df, columna_target, lags=[1, 3, 6, 12]):
    """
    Crea variables de rezago para capturar dependencias temporales
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame con serie temporal
    columna_target : str
        Nombre de la columna objetivo
    lags : list
        Lista de rezagos a crear (en número de periodos)
    
    Returns:
    --------
    DataFrame con nuevas columnas de rezago
    """
    df_lags = df.copy()
    
    for lag in lags:
        df_lags[f'recaudo_lag{lag}'] = df_lags[columna_target].shift(lag)
        print(f"✅ Creado: recaudo_lag{lag} (rezago de {lag} mes(es))")
    
    return df_lags

# TODO: Aplicar función una vez se tenga df mensual
print("⏳ Pendiente: Crear lags 1, 3, 6, 12 meses")

## 4. Rolling Statistics (Ventanas Móviles)

Capturar tendencias y volatilidad en diferentes horizontes

In [ ]:
def crear_rolling_features(df, columna_target, ventanas=[3, 6, 12]):
    """
    Crea estadísticas de ventanas móviles
    """
    df_rolling = df.copy()
    
    for ventana in ventanas:
        # Media móvil
        df_rolling[f'rolling_mean_{ventana}m'] = df_rolling[columna_target].rolling(window=ventana).mean()
        
        # Desviación estándar móvil (volatilidad)
        df_rolling[f'rolling_std_{ventana}m'] = df_rolling[columna_target].rolling(window=ventana).std()
        
        # Min y Max móvil
        df_rolling[f'rolling_min_{ventana}m'] = df_rolling[columna_target].rolling(window=ventana).min()
        df_rolling[f'rolling_max_{ventana}m'] = df_rolling[columna_target].rolling(window=ventana).max()
        
        print(f"✅ Creadas rolling features para ventana de {ventana} meses")
    
    return df_rolling

print("⏳ Pendiente: Crear rolling features (3, 6, 12 meses)")

## 5. Exponential Weighted Moving Average (EWMA)

Dar mayor peso a observaciones recientes

In [ ]:
def crear_ewma_features(df, columna_target, alphas=[0.3, 0.5, 0.7]):
    """
    Crea features con media móvil exponencial
    """
    df_ewma = df.copy()
    
    for alpha in alphas:
        df_ewma[f'ewma_alpha{alpha}'] = df_ewma[columna_target].ewm(alpha=alpha, adjust=False).mean()
        print(f"✅ Creado EWMA con alpha={alpha}")
    
    return df_ewma

print("⏳ Pendiente: Crear EWMA features")

## 6. Features Temporales (Calendario)

Capturar estacionalidad y patrones cíclicos

In [ ]:
def crear_features_temporales(df, columna_fecha):
    """
    Crea features basados en el calendario
    """
    df_temp = df.copy()
    
    # Asegurar que columna_fecha es datetime
    df_temp[columna_fecha] = pd.to_datetime(df_temp[columna_fecha])
    
    # Features básicos
    df_temp['año'] = df_temp[columna_fecha].dt.year
    df_temp['mes'] = df_temp[columna_fecha].dt.month
    df_temp['trimestre'] = df_temp[columna_fecha].dt.quarter
    df_temp['semestre'] = df_temp[columna_fecha].dt.month.apply(lambda x: 1 if x <= 6 else 2)
    
    # Features cíclicos (sin y cos) para capturar periodicidad
    df_temp['mes_sin'] = np.sin(2 * np.pi * df_temp['mes'] / 12)
    df_temp['mes_cos'] = np.cos(2 * np.pi * df_temp['mes'] / 12)
    
    # One-hot encoding de mes (para modelos que lo requieran)
    for mes in range(1, 13):
        df_temp[f'es_mes_{mes}'] = (df_temp['mes'] == mes).astype(int)
    
    print("✅ Features temporales creados")
    return df_temp

print("⏳ Pendiente: Crear features temporales")

## 7. Regresores Exógenos: Calendario Tributario

**Variables externas clave**:
- Declaración de Renta (abril-mayo)
- ICA - Industria y Comercio (marzo, julio)

In [ ]:
def crear_regresores_tributarios(df, columna_fecha=None):
    """
    Crea variables binarias para periodos tributarios clave
    """
    df_trib = df.copy()
    
    if columna_fecha:
        df_trib[columna_fecha] = pd.to_datetime(df_trib[columna_fecha])
        mes = df_trib[columna_fecha].dt.month
    elif 'mes' in df_trib.columns:
        mes = df_trib['mes']
    else:
        print("⚠️  No se encontró columna de mes")
        return df_trib
    
    # Periodo de Renta (abril-mayo)
    df_trib['es_periodo_renta'] = mes.isin([4, 5]).astype(int)
    
    # Periodo ICA (marzo, julio)
    df_trib['es_periodo_ica'] = mes.isin([3, 7]).astype(int)
    
    # Periodo de cierre fiscal (diciembre)
    df_trib['es_cierre_fiscal'] = (mes == 12).astype(int)
    
    # Periodo post-festivo (enero)
    df_trib['es_enero'] = (mes == 1).astype(int)
    
    print("✅ Regresores tributarios creados:")
    print("   - es_periodo_renta (abril-mayo)")
    print("   - es_periodo_ica (marzo, julio)")
    print("   - es_cierre_fiscal (diciembre)")
    print("   - es_enero (pico post-festivo)")
    
    return df_trib

print("⏳ Pendiente: Crear regresores tributarios")

## 8. Features de Momentum y Aceleración

In [ ]:
def crear_features_momentum(df, columna_target):
    """
    Crea features de cambio y aceleración
    """
    df_mom = df.copy()
    
    # Diferenciación de primer orden (cambio mensual)
    df_mom['diff_1'] = df_mom[columna_target].diff(1)
    
    # Diferenciación de segundo orden (aceleración)
    df_mom['diff_2'] = df_mom[columna_target].diff(1).diff(1)
    
    # Cambio porcentual
    df_mom['pct_change'] = df_mom[columna_target].pct_change()
    
    # Momentum (suma de cambios últimos 3 meses)
    df_mom['momentum_3m'] = df_mom['diff_1'].rolling(window=3).sum()
    
    print("✅ Features de momentum creados")
    return df_mom

print("⏳ Pendiente: Crear features de momentum")

## 9. Pipeline Completo de Feature Engineering

In [ ]:
def pipeline_feature_engineering(df, columna_target, columna_fecha):
    """
    Ejecuta el pipeline completo de feature engineering
    """
    print("="*80)
    print("INICIANDO PIPELINE DE FEATURE ENGINEERING")
    print("="*80)
    
    df_features = df.copy()
    
    # 1. Features temporales
    print("\n🔄 Paso 1: Features Temporales")
    df_features = crear_features_temporales(df_features, columna_fecha)
    
    # 2. Regresores tributarios
    print("\n🔄 Paso 2: Regresores Tributarios")
    df_features = crear_regresores_tributarios(df_features)
    
    # 3. Lags
    print("\n🔄 Paso 3: Variables de Rezago")
    df_features = crear_lags(df_features, columna_target, lags=[1, 3, 6, 12])
    
    # 4. Rolling features
    print("\n🔄 Paso 4: Rolling Statistics")
    df_features = crear_rolling_features(df_features, columna_target, ventanas=[3, 6, 12])
    
    # 5. EWMA
    print("\n🔄 Paso 5: EWMA Features")
    df_features = crear_ewma_features(df_features, columna_target, alphas=[0.3, 0.5, 0.7])
    
    # 6. Momentum
    print("\n🔄 Paso 6: Momentum Features")
    df_features = crear_features_momentum(df_features, columna_target)
    
    print("\n" + "="*80)
    print("✅ PIPELINE COMPLETADO")
    print(f"Features creados: {df_features.shape[1] - df.shape[1]}")
    print(f"Total de columnas: {df_features.shape[1]}")
    print("="*80)
    
    return df_features

print("⏳ Pipeline definido. Ejecutar cuando se tenga df mensual")

## 10. Manejo de Valores Faltantes por Lags

In [ ]:
# Los lags generan NaN en las primeras observaciones
# Estrategia: eliminar filas con NaN (perder primeros 12 meses del histórico)
# O rellenar con estrategias específicas

print("⏳ Estrategia de NaN:")
print("1. Identificar filas con NaN")
print("2. Eliminar primeras filas (lag máximo = 12 meses)")
print("3. Validar que train set tenga suficientes datos")

## 11. Agregaciones Multi-Horizonte

Preparar datasets para los 3 horizontes temporales

### 11.1. Dataset Mensual

In [ ]:
# TODO: Dataset mensual ya está preparado
print("✅ Dataset mensual: Base para horizonte mensual")
print("   Features: Todos los creados en el pipeline")

### 11.2. Dataset Trimestral

In [ ]:
def agregar_trimestral(df_mensual, columna_target, columna_fecha):
    """
    Agrega datos mensuales a trimestral
    """
    df_trim = df_mensual.copy()
    df_trim[columna_fecha] = pd.to_datetime(df_trim[columna_fecha])
    
    # Crear periodo trimestral
    df_trim['periodo_trim'] = df_trim[columna_fecha].dt.to_period('Q')
    
    # Agregar recaudo por trimestre (suma)
    df_trimestral = df_trim.groupby('periodo_trim').agg({
        columna_target: 'sum',
        'año': 'first',
        'trimestre': 'first',
        # Regresores: tomar máximo (si hubo evento en el trimestre)
        'es_periodo_renta': 'max',
        'es_periodo_ica': 'max',
        'es_cierre_fiscal': 'max',
        'es_enero': 'max'
    }).reset_index()
    
    print(f"✅ Dataset trimestral creado: {len(df_trimestral)} trimestres")
    return df_trimestral

print("⏳ Pendiente: Crear agregación trimestral")

### 11.3. Dataset Semestral

In [ ]:
def agregar_semestral(df_mensual, columna_target, columna_fecha):
    """
    Agrega datos mensuales a semestral
    """
    df_sem = df_mensual.copy()
    df_sem[columna_fecha] = pd.to_datetime(df_sem[columna_fecha])
    
    # Crear campo año-semestre
    df_sem['semestre_periodo'] = df_sem['año'].astype(str) + '-S' + df_sem['semestre'].astype(str)
    
    # Agregar recaudo por semestre (suma)
    df_semestral = df_sem.groupby(['año', 'semestre']).agg({
        columna_target: 'sum',
        'es_periodo_renta': 'max',
        'es_periodo_ica': 'max',
        'es_cierre_fiscal': 'max',
        'es_enero': 'max'
    }).reset_index()
    
    # Crear periodo legible
    df_semestral['periodo_sem'] = df_semestral['año'].astype(str) + '-S' + df_semestral['semestre'].astype(str)
    
    print(f"✅ Dataset semestral creado: {len(df_semestral)} semestres")
    return df_semestral

print("⏳ Pendiente: Crear agregación semestral")

## 12. División Train-Test para los 3 Horizontes

In [ ]:
def dividir_train_test_horizonte(df, columna_fecha, año_test=2026):
    """
    Divide datos en train (hasta 2025) y test (2026)
    """
    df_copy = df.copy()
    
    if 'año' not in df_copy.columns:
        df_copy['año'] = pd.to_datetime(df_copy[columna_fecha]).dt.year
    
    train = df_copy[df_copy['año'] < año_test]
    test = df_copy[df_copy['año'] == año_test]
    
    print(f"✅ División completada:")
    print(f"   Train: {len(train)} registros (2022-2025)")
    print(f"   Test:  {len(test)} registros (2026)")
    
    return train, test

print("⏳ Pendiente: Aplicar división a los 3 datasets")

## 13. Exportar Datasets Procesados

In [ ]:
print("💾 EXPORTAR DATASETS:")
print("="*80)

# TODO: Guardar datasets procesados
archivos_exportar = [
    '../data/features/datos_con_features_mensual.csv',
    '../data/features/datos_trimestral.csv',
    '../data/features/datos_semestral.csv',
    '../data/features/train_mensual.csv',
    '../data/features/test_mensual.csv',
    '../data/features/train_trimestral.csv',
    '../data/features/test_trimestral.csv',
    '../data/features/train_semestral.csv',
    '../data/features/test_semestral.csv'
]

for archivo in archivos_exportar:
    print(f"  - {archivo}")

print("\n✅ Datasets listos para modelado")
print("\n📋 SIGUIENTE PASO:")
print("   Abrir cuaderno: 03_Modelos_Predictivos.ipynb")

## 14. Resumen de Features Creados

In [ ]:
print("="*80)
print("RESUMEN DE FEATURE ENGINEERING")
print("="*80)

print("\n📊 CATEGORÍAS DE FEATURES:")
print("\n1. FEATURES TEMPORALES (16):")
print("   - año, mes, trimestre, semestre")
print("   - mes_sin, mes_cos (ciclicidad)")
print("   - es_mes_1 a es_mes_12 (one-hot)")

print("\n2. REGRESORES TRIBUTARIOS (4):")
print("   - es_periodo_renta, es_periodo_ica")
print("   - es_cierre_fiscal, es_enero")

print("\n3. VARIABLES DE REZAGO (4):")
print("   - recaudo_lag1, lag3, lag6, lag12")

print("\n4. ROLLING STATISTICS (12):")
print("   - rolling_mean/std/min/max (3, 6, 12 meses)")

print("\n5. EWMA FEATURES (3):")
print("   - ewma_alpha0.3, alpha0.5, alpha0.7")

print("\n6. MOMENTUM FEATURES (4):")
print("   - diff_1, diff_2, pct_change, momentum_3m")

print("\n✅ TOTAL DE FEATURES INGENIERÍA: ~43 nuevas variables")
print("\n🎯 Listos para entrenamiento de modelos de series de tiempo")
print("="*80)